### Introduction
In this notebook, we will test out pulling out a [nreimers/MiniLM-L6-H384-uncased](https://huggingface.co/nreimers/MiniLM-L6-H384-uncased) and serving with [Infinity](https://michaelfeil.eu/infinity/latest/) in SAP AI Core. Subsequently, we will use the embedding model and work with SAP HANA Cloud Vector Engine to store the embeddings (Note: bge-small-en uses dimension of 384). After which, we will show you how to perform similarity search function as well.

### Prerequisites
Before running this notebook, please assure you have performed the [Prerequisites](../../README.md) and [01-deployment.ipynb](01-deployment.ipynb). As a result, a deployment of Infinity scenario is running in SAP AI Core.<br/><br/>

If the configuration and deployment are created through SAP AI Launchpad, please manually update the configuration_id and deployment_id in [env.json](env.json)
```json
{
    "configuration_id": "<YOUR_CONFIGURATION_ID_OF_INFINITY_SCENARIO>",
    "deployment_id": "<YOUR_DEPLOYMENT_ID_BASED_ON_CONFIG_ABOVE>"
}
```
 
### The high-level flow:
1. Load configurations info
2. Connect to SAP AI Core via SDK
3. Check the status and logs of the deployment
4. Inference the model through **SAP Generative AI Hub SDK** using OpenAI Embeddings
5. Using Infinity to generate embeddings to store in SAP HANA Cloud Vector Engine
6. Perform Similarity Search using Infinity > bge-small-en embedding model with SAP HANA Cloud Vector Engine


#### 1. Load config info 
- resource_group loaded from [config.json](../config.json)
- deployment_id(created in 01-deployment.ipynb) loaded [env.json](env.json)

In [43]:
import requests, json
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

In [44]:
# Please replace the configurations below.
# config_id: The target configuration to create the deployment. Please create the configuration first.
with open("../config.json") as f:
    config = json.load(f)

with open("./env.json") as f:
    env = json.load(f)

deployment_id = env["deployment_id"]
resource_group = config.get("resource_group", "default")
print("deployment id: ", deployment_id, " resource group: ", resource_group)

deployment id:  d98287077fa69df2  resource group:  oss-llm


#### 2. Initiate connection to SAP AI Core 

In [45]:
# Initiate an AI Core SDK client with the information of service key
ai_core_sk = config["ai_core_service_key"]
base_url = ai_core_sk.get("serviceurls").get("AI_API_URL") + "/v2/lm"
ai_core_client = AICoreV2Client(base_url=ai_core_sk.get("serviceurls").get("AI_API_URL")+"/v2",
                        auth_url=ai_core_sk.get("url")+"/oauth/token",
                        client_id=ai_core_sk.get("clientid"),
                        client_secret=ai_core_sk.get("clientsecret"),
                        resource_group=resource_group)

In [46]:
token = ai_core_client.rest_client.get_token()
headers = {
        "Authorization": token,
        'ai-resource-group': resource_group,
        "Content-Type": "application/json"}


#### 3. Check the deployment status 

In [47]:
# Check deployment status before inference request
deployment_url = f"{base_url}/deployments/{deployment_id}"
response = requests.get(url=deployment_url, headers=headers)
resp = response.json()    
status = resp['status']

deployment_log_url = f"{base_url}/deployments/{deployment_id}/logs"
if status == "RUNNING":
        print(f"Deployment-{deployment_id} is running. Ready for inference request")
else:
        print(f"Deployment-{deployment_id} status: {status}. Not yet ready for inference request")
        #retrieve deployment logs
        #{{apiurl}}/v2/lm/deployments/{{deploymentid}}/logs.

        response = requests.get(deployment_log_url, headers=headers)
        print('Deployment Logs:\n', response.text)


Deployment-d98287077fa69df2 is running. Ready for inference request


#### 4. Inference the model through **SAP Generative AI Hub SDK** using OpenAI Embeddings

In [52]:
from gen_ai_hub.proxy.gen_ai_hub_proxy import GenAIHubProxyClient

GenAIHubProxyClient.add_foundation_model_scenario(
    scenario_id="byom-infinity-server",
    config_names="infinity*",
    prediction_url_suffix="/v1/embeddings",
)

proxy_client = GenAIHubProxyClient(ai_core_client = ai_core_client)

In [54]:
from gen_ai_hub.proxy.native.openai import embeddings

response = embeddings.create(
    input="Every decoding is another encoding.",
    model_name="nreimers/MiniLM-L6-H384-uncased",
    encoding_format='base64'
)
print(response.data[0].embedding)

[-0.05885183811187744, 0.028718644753098488, -0.011316396296024323, -0.004794664680957794, -0.030379917472600937, -0.007130316458642483, 0.14092861115932465, 0.02064255252480507, 0.009383726865053177, -0.0002469807513989508, 0.025675717741250992, -0.018306901678442955, 0.004309441428631544, 0.02422827109694481, 0.0011667977087199688, -0.0075086262077093124, -0.024738168343901634, 0.04987109452486038, -0.040396902710199356, -0.05062771216034889, 0.010058104991912842, -0.0056787580251693726, -0.010888741351664066, -0.015699854120612144, -0.0015070708468556404, 0.016217974945902824, 0.0332748107612133, -0.00993474293500185, 0.10711099952459335, -0.18843114376068115, -0.0013528685085475445, 0.014507356099784374, -0.035922978073358536, -0.007368816062808037, -0.0022760266438126564, -0.027994921430945396, -0.031465500593185425, 0.03806125000119209, -0.02016555331647396, 0.0037419768050312996, 0.01665385253727436, 0.03149839863181114, 0.010954534634947777, -0.016612732782959938, 0.01465539075

#### 5. Using Infinity to generate embeddings to store in SAP HANA Cloud Vector Engine

In [36]:
# Import required libraries
import pandas as pd
import hana_ml
print(pd.__version__)
print(hana_ml.__version__)

2.2.1
2.19.24022101


In [37]:
# 5.1 Create NEW Table for this sample
# Connect to SAP HANA Cloud Vector Engine with hana_ml library
# sa-gen-ai-enablement subaccount
# 6d585c73-26bf-4f19-ae2b-54f3d7e01edf.hna0.prod-eu10.hanacloud.ondemand.com

from hana_ml import ConnectionContext
cc= ConnectionContext(
    address='6d585c73-26bf-4f19-ae2b-54f3d7e01edf.hna0.prod-eu10.hanacloud.ondemand.com', 
    port='443', 
    user='DBADMIN', 
    password='zxCV12#$', 
    encrypt=True
    )
print(cc.hana_version())
print(cc.get_current_schema())

4.00.000.00.1712176279 (fa/CE2024.2)
DBADMIN


In [38]:
# 5.2 get a csv file into pandas df
# Note: At the time of testing, ensure that the records are not already in the table.

df = pd.read_csv('./social_data.csv', sep=',', quotechar='"', low_memory=False)
df.head(80)

,ID,PROCESSOR,PROCESSDATE,PROCESSTIME,REPORTEDBY,DECISION,REDDITPOSTID,MAINTENANCENOTIFICATIONID,ADDRESS,LOCATION,...,GENAIDESCRIPTION,PRIORITY,PRIORITYDESC,SENTIMENT,CATEGORY,DATE,TIME,TEXT,EMBEDDING,VECTOR
0,11,processor,2024-3-9,17:55:47,Mosty_2024,unattended,1ap34of,NaN,27-3 Victoria Rd,"51.553842239632296,0.0041263312666776075",...,"An overflowing dustbin at 27-3 Victoria Rd, Lo...",3,medium,NEUTRAL,PUBLIC CLEANLINESS,2024-02-12,15:52:51,Urgent Announcement for Public Attention! Dear...,"[0.018654607236385345, 0.002410407643765211, -...",000600008CD1983CEEF71D3BA42397BBF8A4F73BAACEB4...
1,12,processor,2024-3-9,17:56:23,Mosty_2024,notified,1ao2ui7,10000320.0,High Road,"51.56570052194053,0.015960105979618958",...,The public area in front of Sagenai Park on Hi...,3,medium,NEUTRAL,PUBLIC CLEANLINESS,2024-02-11,07:55:41,Attention Sagenai residents! 📍\n\nI hope this ...,"[0.022811686620116234, 0.01718984916806221, 0....",0006000093DFBA3CBAD18C3C9A70193CCB3DEE3B5A4BC1...
2,13,processor,2024-3-13,16:39:54,Mosty_2024,notified,1aisbj2,10000321.0,27-3 Victoria Rd,"51.553842239632296,0.0041263312666776075",...,"An overflowing dustbin at 27-3 Victoria Rd, Lo...",3,medium,NEUTRAL,PUBLIC CLEANLINESS,2024-02-04,16:50:35,📢 Urgent Report for Public Attention 📢\n\n&amp...,"[0.017875781282782555, -0.001261840108782053, ...",000600003B70923C5464A5BADD7F2CBB511DBF3A87B9C2...
3,14,processor,2024-3-13,16:40:33,jacobtan89,notified,1amjanl,10000322.0,Albert Road,"51.5512072970387,0.009166761817106052",...,There is a bulky waste left in the common area...,3,medium,NEGATIVE,PUBLIC CLEANLINESS,2024-02-09,08:10:53,📢 Hey fellow Sagenai residents! 🏡🌳\n\n&amp;#x2...,"[0.02464723028242588, 0.004297903273254633, 0....",00060000FDE8C93C6DD58C3BF115643BAA62AD3B0D94B9...
4,15,processor,2024-3-13,16:40:50,jacobtan89,notified,1amj9zl,10000323.0,Hollybush Hill,"51.572192085954335,-0.030992827179612173",...,The grass in the beautiful park on Hollybush H...,3,medium,NEUTRAL,FACILITY & PARK MAINTENANCE,2024-02-09,08:09:37,📢 Attention Sagenai residents! 📍\n\n&amp;#x200...,"[0.007574773393571377, 0.004422580823302269, 0...",00060000CE35F83B4CEB903B1616313C3C465DBC687405...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,69,processor,2024-3-14,08:20:32,Trinidad_MG,notified,1aeqluz,10000377.0,High Road and Grove Green Road,"51.56695925174642, -0.006486404268305011",...,A potential mosquito breeding site has been di...,3,medium,NEGATIVE,PESTS,2024-01-30,15:10:51,"🚨URGENT🚨 Hey, Sagenai! \nI'm so thrilled to ...",NaN,NaN
59,70,processor,2024-3-19,20:35:53,BraveDistrict4961,notified,1ads7xz,10000410.0,High Road,"51.56438148195644, -0.012273466333225048",...,A massive fallen tree in the middle of Grove P...,3,medium,NEGATIVE,FACILITY & PARK MAINTENANCE,2024-01-29,10:27:28,🚨🌳NOT SO SAFE PARK ALERT!🌳🚨\n\nHey u/SagenaiCo...,NaN,NaN
60,71,processor,2024-3-19,20:36:32,btpsa,notified,1biuf9w,10000411.0,High Road,"float, float",...,A massive fallen tree in the middle of Grove P...,2,high,NEGATIVE,FACILITY & PARK MAINTENANCE,2024-03-19,20:30:35,**🚨🌳NOT SO SAFE PARK ALERT!🌳🚨Hey** u/SagenaiCo...,NaN,NaN
61,72,processor,2024-3-19,23:44:46,Eddy_2024,notified,1biz4rp,10000412.0,High Road,"float, float",...,"There's a pot hole on High Road, near the junc...",2,high,NEUTRAL,ROADS & FOOTPATHS,2024-03-19,23:40:10,📢 Attention residents of Sagenai! 🚧I hope this...,NaN,NaN


In [39]:
# 5.3 SELECT query to HDI container's table to ensure connection context works

hdf = cc.sql('''SELECT TOP 10 "ID", "TEXT", "VECTOR" FROM SOCIAL_CITIZEN_GENAI_PROCESSEDISSUES''')
df_abstract = hdf.collect()
print(df_abstract)

    ID                                               TEXT  \
0  999  Urgent Announcement for Public Attention! Dear...   
1   11  Urgent Announcement for Public Attention! Dear...   
2   12  Attention Sagenai residents! 📍\n\nI hope this ...   
3   13  📢 Urgent Report for Public Attention 📢\n\n&amp...   
4   14  📢 Hey fellow Sagenai residents! 🏡🌳\n\n&amp;#x2...   
5   15  📢 Attention Sagenai residents! 📍\n\n&amp;#x200...   
6   16  🚨 URGENT FACILITY MAINTENANCE ISSUE IN SAGENAI...   
7   17  🚧 So apparently the local council found it hil...   
8   18  Can we talk about the disgraceful state of our...   
9   19  Can we talk about the absolutely mind-boggling...   

                                              VECTOR  
0  [128, 1, 0, 0, 222, 251, 85, 60, 225, 91, 213,...  
1  [128, 1, 0, 0, 222, 251, 85, 60, 225, 91, 213,...  
2  [128, 1, 0, 0, 27, 80, 8, 187, 61, 112, 9, 189...  
3  [128, 1, 0, 0, 3, 127, 154, 60, 203, 30, 188, ...  
4  [128, 1, 0, 0, 66, 21, 85, 60, 120, 16, 37, 18... 

In [40]:
# 5.4 Embedding method - using open source embedding model

from gen_ai_hub.proxy.native.openai import embeddings
def get_embedding(input, model="nreimers/MiniLM-L6-H384-uncased") -> str: 
    response = embeddings.create(
        model_name ="nreimers/MiniLM-L6-H384-uncased",
        input=input
    )
    # print(response.data[0].embedding)
    return response.data[0].embedding

In [ ]:
# 5.5 Drop table if already exists

# cursor = cc.connection.cursor()
# sql_command = '''DROP TABLE SOCIAL_CITIZEN_GENAI_PROCESSEDISSUES;'''
# cursor.execute(sql_command)
# cursor.close()

In [75]:
# 5.6 Create a table

cursor = cc.connection.cursor()
sql_command = '''CREATE TABLE SOCIAL_CITIZEN_GENAI_PROCESSEDISSUES(ID INTEGER, PROCESSOR NVARCHAR(5000), PROCESSDATE NVARCHAR(5000), PROCESSTIME NVARCHAR(5000), REPORTEDBY NVARCHAR(5000), DECISION NVARCHAR(5000), REDDITPOSTID NVARCHAR(5000), MAINTENANCENOTIFICATIONID NVARCHAR(5000), ADDRESS NVARCHAR(5000), LOCATION NVARCHAR(5000), LAT NVARCHAR(5000), LONG NVARCHAR(5000), GENAISUMMARY NVARCHAR(5000), GENAIDESCRIPTION NVARCHAR(5000), PRIORITY NVARCHAR(5000), PRIORITYDESC NVARCHAR(5000), SENTIMENT NVARCHAR(5000), CATEGORY NVARCHAR(5000), DATE DATE, TIME NVARCHAR(5000), TEXT NVARCHAR(5000), EMBEDDING NCLOB);'''
cursor.execute(sql_command)
cursor.close()

In [76]:
# 5.7 Add REAL_VECTOR column
# Note: here you define the dimension that suits to the model you're using
# e.g. sentence-transformers/all-MiniLM-L6-v2 uses dimension of 384
# https://www.sbert.net/docs/pretrained_models.html

cursor = cc.connection.cursor()
sql_command = '''ALTER TABLE SOCIAL_CITIZEN_GENAI_PROCESSEDISSUES ADD (VECTOR REAL_VECTOR(384));'''
cursor.execute(sql_command)
cursor.close()

In [83]:
# 5.8 prepare rows to be bulk insert
# generate embeddings from the text

import math
rows = []
for index, row in df.iterrows():
    data_to_insert = row.to_dict()
    text=row.TEXT
    x=row.MAINTENANCENOTIFICATIONID

    # check on maintenance notification id as some values are NaN
    if math.isnan(x):
        maintenanceNotID=0
    else:
        maintenanceNotID=row.MAINTENANCENOTIFICATIONID
    
    text_vector = get_embedding(input=text)
    
    myrow = (row['ID'], row['PROCESSOR'], row['PROCESSDATE'], row['PROCESSTIME'], row['REPORTEDBY'], row['DECISION'], 
             row['REDDITPOSTID'], maintenanceNotID, row['ADDRESS'], row['LOCATION'], row['LAT'], 
             row['LONG'], row['GENAISUMMARY'], row['GENAIDESCRIPTION'], row['PRIORITY'], row['PRIORITYDESC'], row['SENTIMENT'], 
             row['CATEGORY'], row['DATE'], row['TIME'], row['TEXT'], str(text_vector), str(text_vector))
    
    rows.append(myrow)
# print(myrow)

In [84]:
# 5.9 bulk insert of 23 fields parameterised in rows variable
# for more details of the dimension for the model, refer to https://huggingface.co/nreimers/MiniLM-L6-H384-uncased
# nreimers/MiniLM-L6-H384-uncased has dimension of 384, thus mismatched with the dimension you've defined for the vector column in SAP HANA Cloud Vector Database

cc.connection.setautocommit(False)
cursor = cc.connection.cursor()
sql = '''INSERT INTO SOCIAL_CITIZEN_GENAI_PROCESSEDISSUES 
VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,TO_REAL_VECTOR(?));'''
try:
    cursor.executemany(sql, rows)
except Exception as e:
    cc.connection.rollback()
    print("An error occurred:", e)
try:
    cc.connection.commit()
finally:
    cursor.close()
cc.connection.setautocommit(True)

#### 6. Perform Similarity Search using Infinity > bge-small-en embedding model with SAP HANA Cloud Vector Engine

In [115]:
# 6.1 Define a run_vector_search with metric and query as input parameters

def run_vector_search(query: str, metric="COSINE_SIMILARITY", k=4):
    if metric == 'L2DISTANCE':
        sort = 'ASC'
    else:
        sort = 'DESC'
    query_vector = get_embedding(input=query)
    sql = '''SELECT TOP {k} "ID", "CATEGORY", "TEXT", "DATE", "LOCATION", "{metric}"("VECTOR", TO_REAL_VECTOR('{qv}')) AS SIM
        FROM "DBADMIN"."SOCIAL_CITIZEN_GENAI_PROCESSEDISSUES"
        ORDER BY "SIM" {sort}'''.format(k=k, metric=metric, qv=query_vector, sort=sort)
    hdf = cc.sql(sql)
    df_context = hdf.head(k).collect()
    return df_context

In [ ]:
# 6.1 Prepare a text string to be used as input for the similarity search

vector_str = """📢 Urgent Report for Public Attention 📢

Dear neighbours of Sagenai,

I hope this post finds you well. I am writing today to bring to your attention a pressing issue that requires immediate action from our local authorities. 🚮

In the heart of our beautiful neighbourhood, specifically at 27-3 Victoria Rd, London, UK, we are currently facing a problem that greatly affects our daily lives: an overflowing dustbin. 🗑️ The pungent odor, unsightly sight, and the potential for vermin and health hazards pose a significant inconvenience for all of us. 🤢

I kindly request our esteemed local administration to address this matter promptly, ensuring the cleanliness and hygiene we deserve in our shared spaces. 🙏🏼 Let's work together to maintain the charm and cleanliness of our beloved Sagenai!

Thank you for your attention and support in resolving this matter.

Best regards,

Concerned Citizen 🌟

Coordinates:(51.553842239632296,0.0041263312666776075)

&amp;#x200B;

https://preview.redd.it/8f6f8tumw"""

In [134]:
# 6.3 Perform the similarity search and print out k number of similar search results

df = run_vector_search(query = vector_str, k = 10)
df

,ID,CATEGORY,TEXT,DATE,LOCATION,SIM
0,26,PUBLIC CLEANLINESS,📢 Urgent Report for Public Attention 📢\n\nDear...,2024-02-03,"51.553842239632296,0.0041263312666776075",1.000000
1,13,PUBLIC CLEANLINESS,📢 Urgent Report for Public Attention 📢\n\n&amp...,2024-02-04,"51.553842239632296,0.0041263312666776075",0.988930
2,40,PUBLIC CLEANLINESS,📢 Attention Sagenai residents! 📢\n\n&amp;#x200...,2024-02-01,"51.564875435684534,0.01679572596184302",0.947843
3,59,PUBLIC CLEANLINESS,📢 Attention fellow Sagenai residents! 🏡\n\n&am...,2024-01-31,"51.57190697091999, -0.028626163737301077",0.944420
4,11,PUBLIC CLEANLINESS,Urgent Announcement for Public Attention! Dear...,2024-02-12,"51.553842239632296,0.0041263312666776075",0.921868
5,999,PUBLIC CLEANLINESS,Urgent Announcement for Public Attention! Dear...,2024-02-12,"51.553842239632296,0.0041263312666776075",0.921868
6,34,PUBLIC CLEANLINESS,📣 Attention fellow Sagenai residents! 📣\n\nI h...,2024-02-02,"51.564562910897315,-0.018637765272000048",0.902331
7,65,PUBLIC CLEANLINESS,Hey everyone! 👋 I wanted to bring up a little ...,2024-01-30,"51.56232633931576,0.004961210312297189",0.901116
8,47,PUBLIC CLEANLINESS,"I hope this message finds you well. 😊 However,...",2024-02-01,"51.5728533044047,-0.03229501754130372",0.895889
9,67,DRAINS & SEWERS,📢 Urgent Public Announcement 📢\n\n&amp;#x200B;...,2024-01-30,"51.563772772936154, -0.010011597090067229",0.894030
